# Introduction

Correctness in our programs is the extent to which our code does what we intend it to do. Rust is designed with a high degree of concern about the correctness of programs, but correctness is complex and not easy to prove. <span style="color:lightgreen">*Rust’s type system shoulders a huge part of this burden, but the type system cannot catch everything. As such, Rust includes support for writing automated software tests.*</span>

# How to Write Tests

<span style="color:skyblue">*Tests are Rust functions that verify that the non-test code is functioning in the expected manner*</span>. The bodies of test functions typically perform these three actions:
- Set up any needed data or state.
- Run the code you want to test.
- Assert the results are what you expect.

## The Anatomy of a Test Function
At its simplest, a test in Rust is a function that’s annotated with the `test` attribute. Attributes are metadata about pieces of Rust code; one example is the `derive` attribute. <span style="color:lightgreen">***To change a function into a test function, add `#[test]` on the line before `fn`***</span>. When you run your tests with the `cargo test` command, Rust builds a test runner binary that runs the annotated functions and reports on whether each test function passes or fails.

Example: run `cargo new adder --lib` will create the below project
```rust
// src/lib.rs

pub fn add(left: usize, right: usize) -> usize {
    left + right
}

#[cfg(test)]
mod tests {
    use super::*;

    #[test]  // this indicates that this is a test function
    fn it_works() {
        let result = add(2, 2);
        assert_eq!(result, 4);
    }
}
```
which you can run with `cargo test`

## Checking Results with the `assert!` Macro

The `assert!` macro, provided by the standard library, is useful when you want to ensure that some condition in a test evaluates to `true`. <span style="color:lightgreen">***We give the `assert!` macro an argument that evaluates to a Boolean***</span>

For example:

```rust
#[derive(Debug)]
struct Rectangle {
    width: u32,
    height: u32,
}

impl Rectangle {
    fn can_hold(&self, other: &Rectangle) -> bool {
        self.width > other.width && self.height > other.height
    }
}

#[cfg(test)]
mod tests {
    use super::*;

    #[test]
    fn larger_can_hold_smaller() {
        let larger = Rectangle {
            width: 8,
            height: 7,
        };
        let smaller = Rectangle {
            width: 5,
            height: 1,
        };

        assert!(larger.can_hold(&smaller));
    }
}#[derive(Debug)]
struct Rectangle {
    width: u32,
    height: u32,
}

impl Rectangle {
    fn can_hold(&self, other: &Rectangle) -> bool {
        self.width > other.width && self.height > other.height
    }
}

#[cfg(test)]
mod tests {
    use super::*;

    #[test]
    fn larger_can_hold_smaller() {
        let larger = Rectangle {
            width: 8,
            height: 7,
        };
        let smaller = Rectangle {
            width: 5,
            height: 1,
        };

        assert!(larger.can_hold(&smaller));
    }
}
```

## Testing Equality with the `assert_eq!` and `assert_ne!` Macros

<span style="color:lightgreen">*The `assert_ne!` macro will pass if the two values we give it are not equal and fail if they’re equal.*</span>

Under the surface, the `assert_eq!` and `assert_ne!` macros use the operators `==` and `!=`, respectively. When the assertions fail, these macros print their arguments using debug formatting, which means the values being compared must implement the `PartialEq` and `Debug` traits. All primitive types and most of the standard library types implement these traits. *For structs and enums that you define yourself, you’ll need to implement `PartialEq` to assert equality of those types.*

## Adding Custom Failure Messages

<span style="color:lightgreen">*You can also add a custom message to be printed with the failure message as optional arguments to the `assert!`, `assert_eq!`, and `assert_ne!` macros*</span>, for example:
```rust
    #[test]
    fn greeting_contains_name() {
        let result = greeting("Carol");
        assert!(
            result.contains("Carol"),
            "Greeting did not contain name, value was `{}`",
            result
        );
    }

```

## Checking for Panics with `should_panic`

<span style="color:lightgreen">*With `#[should_panic]`, the test passes if the code inside the function panics; the test fails if the code inside the function doesn’t panic*</span>. For example:

```rust
pub struct Guess {
    value: i32,
}

impl Guess {
    pub fn new(value: i32) -> Guess {
        if value < 1 || value > 100 {
            panic!("Guess value must be between 1 and 100, got {}.", value);
        }

        Guess { value }
    }
}

#[cfg(test)]
mod tests {
    use super::*;

    #[test]
    #[should_panic]
    fn greater_than_100() {
        Guess::new(200);
    }
}pub struct Guess {
    value: i32,
}

impl Guess {
    pub fn new(value: i32) -> Guess {
        if value < 1 || value > 100 {
            panic!("Guess value must be between 1 and 100, got {}.", value);
        }

        Guess { value }
    }
}

#[cfg(test)]
mod tests {
    use super::*;

    #[test]
    #[should_panic]
    fn greater_than_100() {
        Guess::new(200);
    }
}
```

Tests that use `should_panic` can be imprecise. A `should_panic` test would pass even if the test panics for a different reason from the one we were expecting. To make `should_panic` tests more precise, we can add an optional `expected` parameter to the `should_panic` attribute.
```rust
// --snip--

impl Guess {
    pub fn new(value: i32) -> Guess {
        if value < 1 {
            panic!(
                "Guess value must be greater than or equal to 1, got {}.",
                value
            );
        } else if value > 100 {
            panic!(
                "Guess value must be less than or equal to 100, got {}.",
                value
            );
        }

        Guess { value }
    }
}

#[cfg(test)]
mod tests {
    use super::*;

    #[test]
    #[should_panic(expected = "less than or equal to 100")]
    fn greater_than_100() {
        Guess::new(200);
    }
}
```

## Using `Result<T, E>` in Tests
We can also write tests that use `Result<T, E>` (and not just using `panic!`)

```rust
#[cfg(test)]
mod tests {
    #[test]
    fn it_works() -> Result<(), String> {
        if 2 + 2 == 4 {
            Ok(())
        } else {
            Err(String::from("two plus two does not equal four"))
        }
    }
}
```
<span style="color:lightgreen">*In the body of the function, rather than calling the `assert_eq!` macro, we return `Ok(())` when the test passes and an `Err` with a `String` inside when the test fails.*</span>

# Controlling How Tests Are Run
<span style="color:lightgreen">*Just as `cargo run` compiles your code and then runs the resulting binary, `cargo test` compiles your code in test mode and runs the resulting test binary. The default behavior of the binary produced by `cargo test` is to run all the tests in parallel and capture output generated during test runs, preventing the output from being displayed and making it easier to read the output related to the test results. You can, however, specify command line options to change this default behavior.*</span>



## Running Tests in Parallel or Consecutively

<span style="color:orange">*When you run multiple tests, by default they run in parallel using threads, meaning they finish running faster and you get feedback quicker. Because the tests are running at the same time, you must make sure your tests don’t depend on each other or on any shared state, including a shared environment, such as the current working directory or environment variables.*</span>

You can also only run with 1 thread (no parallelism), or specify the number of threads with `--test-threads`:  

```bash
$ cargo test -- --test-threads=1
```

## Showing Function Output

By default, if we call `println!` in a test and the test passes, we won’t see the `println!` output in the terminal; we’ll see only the line that indicates the test passed. If a test fails, we’ll see whatever was printed to standard output with the rest of the failure message.

If we want to see printed values for passing tests as well, we can tell Rust to also show the output of successful tests with `--show-output`.
```bash
$ cargo test -- --show-output
```

## Running a Subset of Tests by Name

For example, if we have the following tests

```rust
pub fn add_two(a: i32) -> i32 {
    a + 2
}

#[cfg(test)]
mod tests {
    use super::*;

    #[test]
    fn add_two_and_two() {
        assert_eq!(4, add_two(2));
    }

    #[test]
    fn add_three_and_two() {
        assert_eq!(5, add_two(3));
    }

    #[test]
    fn one_hundred() {
        assert_eq!(102, add_two(100));
    }
}
```

- Run all 3 tests with `cargo test`  
- Run a single test with `cargo test one_hundred`  
- Run multiple tests with `cargo test add` will run the `add_two_and_two` and `add_three_and_two` tests

## Ignoring Some Tests Unless Specifically Requested

You can use the `ignore` attribute like below to ignore a test

```rust
#[test]
fn it_works() {
    assert_eq!(2 + 2, 4);
}

#[test]
#[ignore]
fn expensive_test() {
    // code that takes an hour to run
}
```
If we want to run the ignored tests, we can use
```bash
cargo test -- --ignored
```


# Test Organization

The Rust community thinks about tests in terms of two main categories: unit tests and integration tests.  
- <span style="color:green">*Unit tests are small and more focused, testing one module in isolation at a time, and can test private interfaces.*</span>    
- <span style="color:green">*Integration tests are entirely external to your library and use your code in the same way any other external code would, using only the public interface and potentially exercising multiple modules per test.*</span>

## Unit Tests
The purpose of unit tests is to test each unit of code in isolation from the rest of the code to quickly pinpoint where code is and isn’t working as expected.

<span style="color:skyblue">*The convention is to create a module named tests in each file to contain the `test` functions and to annotate the module with `cfg(test)`.*</span>

### The Tests Module and `#[cfg(test)]`
The `#[cfg(test)]` annotation on the tests module tells Rust to compile and run the test code only when you run `cargo test`, not when you run `cargo build` to save compile time. 

```rust
#[cfg(test)]
mod tests {
    #[test]
    fn it_works() {
        let result = 2 + 2;
        assert_eq!(result, 4);
    }
}
```

For integration tests, since they go in a different directory, they don’t need the `#[cfg(test)]` annotation.

### Testing Private Functions

Rust allows us to test private functions, e.g. the `internal_adder` function below is not marked as `pub`

```rust
pub fn add_two(a: i32) -> i32 {
    internal_adder(a, 2)
}

fn internal_adder(a: i32, b: i32) -> i32 {
    a + b
}

#[cfg(test)]
mod tests {
    use super::*;

    #[test]
    fn internal() {
        assert_eq!(4, internal_adder(2, 2));
    }
}
```

## Integration Tests
The purpose of integration tests is to test whether many parts of your library work together correctly. To create integration tests, you first need a `tests` directory.

```bash
adder
├── Cargo.lock
├── Cargo.toml
├── src
│   └── lib.rs
└── tests
    └── integration_test.rs
```
And then in `tests/integration_test.rs`:
```rust
use adder;

#[test]
fn it_adds_two() {
    assert_eq!(4, adder::add_two(2));
}
```

Each file in the tests directory is a separate crate, so we need to bring our library into each test crate’s scope. For that reason we add `use adder` at the top of the code, which we didn’t need in the unit tests.

<span style="color:orange">*When we run `cargo test`, it will first run the unit tests, integration test, and doc tests. If any test in a section fails, the following sections will not be run*</span>

We can still run a particular integration test function by specifying the test function’s name as an argument to cargo test. To run all the tests in a particular integration test file, use the `--test` argument of `cargo test` followed by the name of the file:

```bash
cargo test --test integration_test
```

### Submodules in Integration Tests

If we create `tests/common.rs` and place a function named `setup` in it, we can add some code to `setup` that we want to call from multiple test functions in multiple test files. However, since we don't have any test functions in `tests/common.rs`, we should rename it to `tests/common/mod.rs`

```bash
├── Cargo.lock
├── Cargo.toml
├── src
│   └── lib.rs
└── tests
    ├── common
    │   └── mod.rs
    └── integration_test.rs
```
Naming the file this way tells Rust not to treat the `common` module as an integration test file. Files in subdirectories of the tests directory don’t get compiled as separate crates or have sections in the test output.

After we’ve created `tests/common/mod.rs`, we can use it from any of the integration test files as a module

### Integration Tests for Binary Crates
<span style="color:green">***If our project is a binary crate that only contains a `src/main.rs` file and doesn’t have a `src/lib.rs` file, we can’t create integration tests in the tests directory and bring functions defined in the `src/main.rs` file into scope with a use statement. Only library crates expose functions that other crates can use; binary crates are meant to be run on their own.***</span>